<a href="https://colab.research.google.com/github/Bladefinger/100DaysOfCode/blob/main/Dynamic_Hand_Gesture_Classification_on_DHG_14_28_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! mkdir dataset_shrec2017
#! wget http://www-rech.telecom-lille.fr/shrec2017-hand/HandGestureDataset_SHREC2017.tar.gz 
#! -O SHREC2017.tar.gz tar -xzf SHREC2017.tar.gz -C dataset_shrec2017

In [ ]:
! mkdir dataset_dhg1428
! wget http://www-rech.telecom-lille.fr/DHGdataset/DHG2016.zip 
! unzip DHG2016.zip -d dataset_dhg1428

In [ ]:
#! mkdir dataset_onlinedhg 
#! wget http://www-rech.telecom-lille.fr/shrec2017-hand/OnlineDHG.zip
#! unzip OnlineDHG.zip -d dataset_onlinedhg

In [ ]:
import glob
import numpy
import pickle
from scipy import ndimage as ndimage
from sklearn.model_selection import train_test_split


def resize_gestures(input_gestures, final_length=100):
    """
    Resize the time series by interpolating them to the same length

    Input:
        - input_gestures: list of numpy.ndarray tensors.
              Each tensor represents a single gesture.
              Gestures can have variable durations.
              Each tensor has a shape: (duration, channels)
              where duration is the duration of the individual gesture
                    channels = 44 = 2 * 22 if recorded in 2D and
                    channels = 66 = 3 * 22 if recorded in 3D 
    Output:
        - output_gestures: one numpy.ndarray tensor.
              The output tensor has a shape: (records, final_length, channels)
              where records = len(input_gestures)
                   final_length is the common duration of all gestures
                   channels is the same as above 
    """
    # please use python3. if you still use python2, important note: redefine the classic division operator / by importing it from the __future__ module
    output_gestures = numpy.array([numpy.array([ndimage.zoom(x_i.T[j], final_length / len(x_i), mode='reflect') for j in range(numpy.size(x_i, 1))]).T for x_i in input_gestures])
    return output_gestures


def load_gestures(dataset='dhg', root='./dataset_dhg1428', version_x='3D', version_y='14', resize_gesture_to_length=100):
    """
    Get the 3D or 2D pose gestures sequences, and their associated labels.

    Ouput:
        - a tuple of (gestures, labels) or (gestures, labels_14, labels_28)
              where gestures is either a numpy.ndarray tensor or
                                       a list of numpy.ndarray tensors,
                                       depending on if the gestures have been resized or not.
              Each tensor represents a single gesture.
              Gestures can have variable durations.
              Each tensor has a shape: (duration, channels) where channels is either 44 (= 2 * 22) or 66 (=3 * 22)
    """

    # SHREC 2017 (on Google Colab):
    # root = '/content/dataset_shrec2017/HandGestureDataset_SHREC2017'
    # DHG 14/28 (on Google Colab):
    root = '/content/dataset_dhg1428'
    if dataset == 'dhg':
      assert 'dataset_dhg' in root
    if dataset == 'shrec':
      assert 'dataset_shrec' in root
    
    if version_x == '3D':
        if dataset == 'dhg':
            pattern = root + '/gesture_*/finger_*/subject_*/essai_*/skeleton_world.txt'
        elif dataset == 'shrec':
            pattern = root + '/gesture_*/finger_*/subject_*/essai_*/skeletons_world.txt'
    else:
        if dataset == 'dhg':
            pattern = root + '/gesture_*/finger_*/subject_*/essai_*/skeleton_image.txt'
        elif dataset == 'shrec':
            pattern = root + '/gesture_*/finger_*/subject_*/essai_*/skeletons_image.txt'

    gestures_filenames = sorted(glob.glob(pattern))
    gestures = [numpy.genfromtxt(f) for f in gestures_filenames]
    if resize_gesture_to_length is not None:
        gestures = resize_gestures(gestures, final_length=resize_gesture_to_length)

    labels_14 = [int(filename.split('/')[-5].split('_')[1]) for filename in gestures_filenames]
    #labels_28 = [int(filename.split('/')[-4].split('_')[1]) for filename in gestures_filenames]
    #labels_28 = [labels_14[idx_gesture] if n_fingers_used == 1 else 14 + labels_14[idx_gesture] for idx_gesture, n_fingers_used in enumerate(labels_28)]

    if version_y == '14' or version_y == 14:
        return gestures, labels_14
    elif version_y == '28' or version_y == 28:
        return gestures, labels_28
    elif version_y == 'both':
        return gestures, labels_14, labels_28


def write_data(data, filepath):
    """Save the dataset to a file. Note: data is a dict with keys 'x_train', ..."""
    with open(filepath, 'wb') as output_file:
        pickle.dump(data, output_file)


def load_data(filepath='./shrec_data.pckl'):
    """
    Returns hand gesture sequences (X) and their associated labels (Y).
    Each sequence has two different labels.
    The first label  Y describes the gesture class out of 14 possible gestures (e.g. swiping your hand to the right).
    The second label Y describes the gesture class out of 28 possible gestures (e.g. swiping your hand to the right with your index pointed, or not pointed).
    """
    file = open(filepath, 'rb')
    data = pickle.load(file, encoding='latin1')  # <<---- change to 'latin1' to 'utf8' if the data does not load
    file.close()
    return data['x_train'], data['x_test'], data['y_train_14'], data['y_test_14']
    #return data['x_train'], data['x_test'], data['y_train_14'], data['y_train_28'], data['y_test_14'], data['y_test_28']


In [ ]:
# ---------------------------------------------------------
# Step 3. Save the dataset(s) you need
# ---------------------------------------------------------
# Example A: 2D version of the SHREC gestures, untouched, and only the 14-label version of the labels
# x_2d_shrec, y_shrec_14 = load_gestures(dataset='shrec',
#                                        root='/tmp/dataset_shrec2017/HandGestureDataset_SHREC2017/',
#                                        version_x='2D',
#                                        version_y='14',
#                                        resize_gesture_to_length=None)

# Example B: 3D version of the DHG gestures, resized to 100 timesteps
#gestures, labels_14, labels_28 = load_gestures(dataset='dhg',
gestures, labels_14 = load_gestures(dataset='dhg',
                                               root='/content/dataset_dhg1428',
                                               version_x='3D',
                                               version_y='14',
                                               resize_gesture_to_length=100)

# Split the dataset into train and test sets if you want:
x_train, x_test, y_train_14, y_test_14 = train_test_split(gestures, labels_14)
#x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = train_test_split(gestures, labels_14, labels_28)
#x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = train_test_split(gestures, labels_14, labels_28, test_size=0.50)
#x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = train_test_split(gestures, labels_14, labels_28, test_size=0.50)

# Save the dataset
data = {
    'x_train': x_train,
    'x_test': x_test,
    'y_train_14': y_train_14,
    #'y_train_28': y_train_28,
    'y_test_14': y_test_14,
    #'y_test_28': y_test_28
}
write_data(data, filepath='dhg_data.pckl')

In [ ]:
#x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = load_data('dhg_data.pckl')
#x_train, x_test, y_train_14, y_test_14, = load_data('dhg_data.pckl')
x_train, x_test, y_train, y_test = load_data('dhg_data.pckl')


In [ ]:
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
#from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py
## input layer
input_layer = Input((210, 22, 3))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
 
## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=10, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
#from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py
## input layer
input_layer = Input((100, 66, 1))

## convolutional layers
conv_layer1 = Conv2D(filters=8, kernel_size=(3, 3), activation='relu')(input_layer)
output_layer = Dense(units=10, activation='softmax')(conv_layer1)


## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.compile(loss=categorical_crossentropy, metrics=['acc'])
model.summary()


In [ ]:
#Load datas

dataset = load_data('dhg_data.pckl')


In [ ]:


# Splitting the dataset for training and testing.
def is_test(x, _):
    return x % 4 == 0


def is_train(x, y):
    return not is_test(x, y)


recover = lambda x, y: y

# Split the dataset for training.
test_dataset = dataset.enumerate() \
    .filter(is_test) \
    .map(recover)

# Split the dataset for testing/validation.
train_dataset = dataset.enumerate() \
    .filter(is_train) \
    .map(recover)

In [ ]:
x_train = np.array(x_train)
X_train = x_train.reshape(323)


In [ ]:
y_train = np.array(y_train)

In [ ]:
y_train = y_train.reshape((100, 66))

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=10, epochs=5, validation_split=0.2)
#model.fit(x=x_train, y=y_train, batch_size=128, epochs=50)